# 신경망 관련 추가 학습
(참고 : 파이토치튜토리얼 - 예제로 배우는 파이토치)

ReLU 신경망을 예제로 사용한다. 이 신경망은 하나의 은닉층 (hidden layer)을 갖고 있으며, 신경망의 출력과 정답 사이의 유클리드 거리 (Euclidean distance)를 최소화하는 식으로 경사하강법(gradient descent)을 사용하여 무작위의 데이터를 맞추도록 학습한다. 

여러 가지 패키지를 이용해 다양한 방법으로 신경망을 구성해보겠다.

### NumPy를 사용한 신경망을 구성해보자.

NumPy는 N차원 배열 객체과 같은 배열들을 조작하기 위한 다양한 함수를 제공한다. NumPy 연산을 사용하여 순전파 단계와 역전파 단계를 직접 구현함으로써, 2계층(two-layer)을 갖는 신경망이 무작위의 데이터를 맞추도록 할 수 있다

In [1]:
 #-*- coding: utf-8 -*-
import numpy as np

# N은 배치 크기이며, D_in은 입력의 차원
# H는 은닉층의 차원이며, D_out은 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

# 무작위의 입력과 출력 데이터를 생성
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# 무작위로 가중치를 초기화
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: 예측값 y를 계산
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # 손실(loss)을 계산하고 출력
    loss = np.square(y_pred - y).sum()
    print(t, loss)

    # 손실에 따른 w1, w2의 변화도를 계산하고 역전파
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # 가중치를 갱신
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 33161658.239218708
1 27195940.433474064
2 24332029.525697783
3 20841073.983486116
4 16175364.396635894
5 11178939.188116077
6 7141376.577036583
7 4409297.190405185
8 2777646.6525287125
9 1837684.8475821982
10 1295548.895976421
11 968909.6994100075
12 760207.4930175476
13 617747.3716856532
14 514592.63701045635
15 436042.11928309646
16 373953.74092161347
17 323502.5410496566
18 281796.40595202695
19 246828.0175246825
20 217166.81248066502
21 191837.0827962893
22 170104.1383574393
23 151304.2463714442
24 134951.23337602918
25 120670.61042552782
26 108179.86522879527
27 97192.22983303432
28 87498.20520992175
29 78926.33207727775
30 71323.35018585651
31 64564.29044866665
32 58538.73716074727
33 53157.52899543967
34 48339.22870816703
35 44015.24819262219
36 40126.519264089104
37 36623.3619332516
38 33465.52410965476
39 30614.086451309868
40 28033.048837121933
41 25696.117279584127
42 23575.771418339977
43 21648.804623230375
44 19895.735346577138
45 18299.298844652032
46 16843.214340894836

443 2.7039554532291e-06
444 2.568318567502596e-06
445 2.4394625641650006e-06
446 2.317128840683463e-06
447 2.200929298383284e-06
448 2.090573498666831e-06
449 1.9857828100447913e-06
450 1.8862599133831183e-06
451 1.7917445102443659e-06
452 1.701949454843139e-06
453 1.6166906318374065e-06
454 1.5357185818938653e-06
455 1.4587980727387725e-06
456 1.385749183168261e-06
457 1.3163477122590708e-06
458 1.25044264735104e-06
459 1.1878747526932012e-06
460 1.128436579973763e-06
461 1.0720121016017676e-06
462 1.0183795203003629e-06
463 9.674248908053798e-07
464 9.190347907690581e-07
465 8.730824804061048e-07
466 8.294355154074555e-07
467 7.879794509117059e-07
468 7.485836035567512e-07
469 7.111715392250751e-07
470 6.756266510321775e-07
471 6.418676357416565e-07
472 6.098019479560899e-07
473 5.793464046254369e-07
474 5.504135094356608e-07
475 5.229252632337216e-07
476 4.968137770319276e-07
477 4.7201030375144606e-07
478 4.484520211455931e-07
479 4.260707641569098e-07
480 4.0481724026102473e-07
48

### Tensor를 사용한 신경망을 구성해보자.

NumPy는 훌륭한 프레임워크지만, GPU를 사용하여 수치 연산을 가속화할 수는 없다. 현대의 심층 신경망에서 GPU는 종종 50배 또는 그 이상 의 속도 향상을 제공하기 때문에,  NumPy는 현대의 딥러닝에 적합하지 않다.

PyTorch Tensor는 개념적으로 NumPy 배열과 동일하다. Tensor는 N차원 배열이며, PyTorch는 Tensor 연산을 위한 다양한 함수들을 제공한다. NumPy와는 달리, PyTorch Tensor는 GPU를 활용하여 수치 연산을 가속화할 수 있다. GPU에서 PyTorch Tensor를 실행하기 위해서는 단지 새로운 자료형으로 변환(Cast)해주기만 하면 된다.

여기에서는 PyTorch Tensor를 사용하여 2계층의 신경망이 무작위 데이터를 맞추도록 할 것이다. 위의 NumPy 예제에서와 같이 신경망의 순전파 단계와 역전파 단계는 직접 구현한다.

In [2]:
# -*- coding: utf-8 -*-
import torch

dtype = torch.float # dtype = 데이터 타입
device = torch.device("cpu") # CPU에서 실행한다는 의미
# device = torch.device("cuda:0") # GPU에서 실행시 이 코드 사용

# N은 배치 크기이며, D_in은 입력의 차원
# H는 은닉층의 차원이며, D_out은 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

# 무작위의 입력과 출력 데이터를 생성
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 무작위로 가중치를 초기화
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: 예측값 y를 계산
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # 손실(loss)을 계산하고 출력
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # 손실에 따른 w1, w2의 변화도를 계산하고 역전파
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # 경사하강법(gradient descent)를 사용하여 가중치를 갱신
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 513.4201049804688
199 2.68537974357605
299 0.02145102247595787
399 0.0003887308994308114
499 5.218850856181234e-05


### Autograd를 사용한 신경망을 구성해보자.

위의 두 예제에서 신경망의 순전파 단계와 역전파 단계를 직접 구현하였다. 대규모의 복잡한 신경망에서는 이를 직접 구현하는것이 복잡하고 어렵다.

다행히, Autograd를 사용하여 신경망에서 역전파 단계의 연산을 자동화할 수 있다. Autograd를 사용할 때, 신경망의 순전파 단계는 연산 그래프를 정의하게 된다. 이 그래프의 노드(node)는 Tensor가 되고, 엣지(edge)는 입력 Tensor로부터 출력 Tensor를 만들어내는 함수가 된다. 이 그래프를 통해 역전파를 하게 되면 변화도를 쉽게 계산할 수 있다.

여기에서는 PyTorch Tensor와 autograd를 사용하여 2계층 신경망을 구현한다. 이제 신경망의 역전파 단계를 직접 구현할 필요가 없다.

In [3]:
# -*- coding: utf-8 -*-
import torch

dtype = torch.float
device = torch.device("cpu")

N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성
# requires_grad=False로 설정하여 역전파 중에 이 Tensor들에 대한 변화도를 계산할
# 필요가 없음을 나타낸다. (requres_grad의 기본값이 False이므로 아래 코드에는
# 이를 반영하지 않았다.)
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 가중치를 저장하기 위해 무작위 값을 갖는 Tensor를 생성
# requires_grad=True로 설정하여 역전파 중에 이 Tensor들에 대한
# 변화도를 계산할 필요가 있음을 나타낸다.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: Tensor 연산을 사용하여 예상되는 y 값을 계산
    # 이는 Tensor를 사용한 순전파 단계와 완전히 동일하지만, 역전파 단계를 별도로
    # 구현하지 않아도 되므로 중간값들에 대한 참조를 갖고 있을 필요가 없다.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # Tensor 연산을 사용하여 손실을 계산하고 출력
    # loss는 (1,) 형태의 Tensor이며, loss.item()은 loss의 스칼라 값
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # autograd를 사용하여 역전파 단계를 계산
    # 이는 requires_grad=True를 갖는 모든 Tensor에 대해 손실의 변화도를 계산
    # w1.grad와 w2.grad는 w1과 w2 각각에 대한 손실의 변화도를 갖는 Tensor가 된다.
    loss.backward()

    # 경사하강법(gradient descent)을 사용하여 가중치를 수동으로 갱신
    # torch.no_grad()로 감싸는 이유는 가중치들이 requires_grad=True이지만
    # autograd에서는 이를 추적할 필요가 없기 때문이다.
    # tensor.data가 tensor의 저장공간을 공유하지만, 이력을 추적하지 않는다는 것을 
    # 기억하고, 이를 위해 torch.optim.SGD 를 사용할 수도 있습니다.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 가중치 갱신 후에는 수동으로 변화도를 0으로 만든다.
        w1.grad.zero_()
        w2.grad.zero_()

99 256.0006103515625
199 0.6370574831962585
299 0.0030558237340301275
399 0.00010187493899138644
499 2.1627420210279524e-05


### nn패키지를 사용하여 신경망을 구성해보자.

연산 그래프와 autograd는 복잡한 연산자를 정의하고 도함수(derivative)를 자동으로 계산하는 매우 강력한 패러다임이다. 하지만 규모가 큰 신경망에서는 autograd 그 자체만으로는 너무 낮은 수준(low-level)일 수 있다.

신경망을 구성할 때 종종 연산을 여러 계층에 배열(arrange)하는 것으로 생각하는데, 이 중 일부는 학습 도중 최적화가 될 학습 가능한 매개변수를 갖고 있다.

PyTorch에서 nn 패키지가 연산 그래프를 더 높은 수준으로 추상화(higher-level abstraction)하여 제공하므로 신경망을 구축하는데 있어 유용하다. nn 패키지는 신경망 계층(layer)들과 거의 동일한 Module의 집합을 정의한다. Module은 입력 Tensor를 받고 출력 Tensor를 계산하며, 학습 가능한 매개변수를 갖는 Tensor 같은 내부 상태(internal state)를 갖는다. 또 nn 패키지는 신경망을 학습시킬 때 주로 사용하는 유용한 손실 함수들도 정의하고 있다.

In [4]:
# -*- coding: utf-8 -*-
import torch

N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위한 Tensor 생성
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# nn 패키지를 사용하여 모델을 순차적 계층(sequence of layers)으로 정의
# nn.Sequential은 다른 Module들을 포함하는 Module로, 그 Module들을 순차적으로
# 적용하여 출력을 생성한다. 각각의 Linear Module은 선형 함수를 사용하여
# 입력으로부터 출력을 계산하고, 내부 Tensor에 가중치와 편향을 저장한다.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# 평균 제곱 오차(MSE; Mean Squared Error)를 손실 함수로 사용
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y 값을 계산한다. Module 객체는
    # __call__ 연산자를 덮어써(override) 함수처럼 호출할 수 있게 한다.
    # 이렇게 함으로써 입력 데이터의 Tensor를 Module에 전달하여 출력 데이터의
    # Tensor를 생성한다.
    y_pred = model(x)

    # 손실을 계산하고 출력
    # 예측한 y와 정답인 y를 갖는 Tensor들을 전달하고, 손실 값을 갖는 Tensor를 반환
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 역전파 단계를 실행하기 전에 변화도를 0으로 만든다.
    model.zero_grad()

    # 역전파 단계: 모델의 모든 학습 가능한 매개변수의 변화도를 계산
    loss.backward()

    # 경사하강법(gradient descent)를 사용하여 가중치를 갱신
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 2.834005355834961
199 0.03594784438610077
299 0.0008381842053495347
399 2.599265644676052e-05
499 9.642888016969664e-07


### optim패키지를 사용하여 신경망을 구성해보자.

지금까지는 (autograd의 추적 기록을 피하기 위해 torch.no_grad () 또는 .data를 사용하는 식으로) 학습 가능한 매개변수를 갖는 Tensor를 직접 조작하며 모델의 가중치를 갱신하였다. 이것은 확률적 경사 하강법(SGD)과 같은 간단한 최적화 알고리즘에서는 크게 부담이 되지 않지만, 실제로 신경망을 학습할 때는 주로 AdaGrad, RMSProp, Adam 등과 같은 좀 더 정교한 Optimizer를 사용한다.

PyTorch의 optim 패키지는 최적화 알고리즘에 대한 아이디어를 추상화하고 일반적으로 사용하는 최적화 알고리즘의 구현체(implementation)를 제공한다.

아래 예제에서는 nn 패키지를 사용하여 모델을 정의하고, optim 패키지가 제공하는 Adam 알고리즘을 이용하여 모델을 최적화한다.

In [5]:
# -*- coding: utf-8 -*-
import torch

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# nn 패키지를 사용하여 모델과 손실 함수를 정의
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# optim 패키지를 사용하여 모델의 가중치를 갱신할 Optimizer를 정의
# Adam을 사용, Adam 생성자의 첫번째 인자는 어떤 Tensor가 갱신되어야 하는지
# 알려준다.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(500):
    # 순전파 단계
    y_pred = model(x)

    # 손실을 계산하고 출력
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 역전파 단계 전에, Optimizer 객체를 사용하여 (모델의 학습 가능한 가중치인)
    # 갱신할 변수들에 대한 모든 변화도를 0으로 만든다. 이렇게 하는 이유는
    # 기본적으로 .backward()를 호출할 때마다 변화도가 버퍼(buffer)에 (덮어쓰지 않고)
    # 누적되기 때문이다.
    optimizer.zero_grad()

    # 역전파 단계
    loss.backward()

    # Optimizer의 step 함수를 호출하면 매개변수가 갱신된다.
    optimizer.step()

99 59.43168258666992
199 1.1043533086776733
299 0.007197318598628044
399 3.527702938299626e-05
499 7.885238062499411e-08


### nn.Module 클래스로 신경망을 구성해보자.

기존 모듈의 구성(sequence)보다 더 복잡한 모델을 구성해야 할 때가 있다. 이럴 때는 nn.Module의 서브클래스로 새 모듈을 정의하고, 입력 Tensor를 받아 다른 모듈 또는 Tensor의 autograd 연산을 사용하여 출력 Tensor를 만드는 forward를 정의하면 된다.

In [6]:
# -*- coding: utf-8 -*-
import torch

class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        생성자에서 2개의 nn.Linear 모듈을 생성하고, 멤버 변수로 지정
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        순전파 함수에서는 입력 데이터의 Tensor를 받고 출력 데이터의 Tensor를
        반환해야 한다. Tensor 상의 임의의 연산자뿐만 아니라 생성자에서 정의한
        Module도 사용할 수 있다.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 앞에서 정의한 클래스를 생성하여 모델을 구성
model = TwoLayerNet(D_in, H, D_out)

# 손실 함수와 Optimizer를 생성한다. SGD 생성자에 model.parameters()를 호출하면
# 모델의 멤버인 2개의 nn.Linear 모듈의 학습 가능한 매개변수들이 포함됩니다.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    # 순전파 단계
    y_pred = model(x)

    # 손실을 계산 후 출력
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 변화도를 0으로 만들고, 역전파 단계를 수행 후 가중치 갱신
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 2.141716480255127
199 0.02532978728413582
299 0.0005677043809555471
399 1.9873172277584672e-05
499 9.447001048101811e-07


### 제어 흐름(Control Flow) + 가중치 공유(Weight Sharing)가 되는 모델로 신경망을 구성해보자.

각 순전파 단계에서 많은 은닉 계층을 갖는 완전히 연결(fully-connected)된 ReLU 신경망이 무작위로 0 ~ 3 사이의 숫자를 선택하고, 가장 안쪽(innermost)의 은닉층들을 계산하기 위해 동일한 가중치를 여러 번 재사용한다.

이 모델에서는 일반적인 Python 제어 흐름을 사용하여 반복(loop)을 구현할 수 있으며, 순전파 단계를 정의할 때 단지 동일한 Module을 여러번 재사용함으로써 내부(innermost) 계층들 간의 가중치 공유를 구현할 수 있다.

Module을 상속받는 서브클래스로 이를 구현해보겠다

In [7]:
# -*- coding: utf-8 -*-
import random
import torch


class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        생성자에서 순전파 단계에서 사용할 3개의 nn.Linear 인스턴스를 생성
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        모델의 순전파 단계에서, 무작위로 0, 1, 2 또는 3 중에 하나를 선택하고
        은닉층을 계산하기 위해 여러번 사용한 middle_linear Module을 재사용한다.

        각 순전파 단계는 동적 연산 그래프를 구성하기 때문에, 모델의 순전파 단계를
        정의할 때 반복문이나 조건문과 같은 일반적인 Python 제어 흐름 연산자를 사용할
        수 있다.

        여기에서 연산 그래프를 정의할 때 동일 Module을 여러번 재사용하는 것이
        완벽히 안전하다는 것을 알 수 있다. 이것이 각 Module을 한 번씩만 사용할
        수 있었던 Lua Torch보다 크게 개선된 부분이다.
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred

    
N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 앞서 정의한 클래스를 생성(instantiating)하여 모델을 구성
model = DynamicNet(D_in, H, D_out)

# 손실함수와 Optimizer를 생성한다. 이 모델은 순수한 확률적 경사 하강법
# (stochastic gradient decent)으로 학습하는 것은 어려우므로, 모멘텀(momentum)을
# 사용한다.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    # 순전파 단계
    y_pred = model(x)

    # 손실을 계산 후 출력
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 변화도를 0으로 만들고, 역전파 단계를 수행한 후 가중치 갱신
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 19.300527572631836
199 18.16835594177246
299 0.6223381757736206
399 0.3568560779094696
499 0.6921087503433228
